# Intelligent Agents #

This notebook serves as supporting material for topics covered in **Chapter 2 - Intelligent Agents** from the book *Artificial Intelligence: A Modern Approach.* 

## Learning Objectives
By the end of this notebook, you will understand:
- The formal definition of an intelligent agent and its components
- How agents perceive and act in environments
- The agent-environment interaction loop
- How to implement simple and complex agent behaviors
- The progression from reactive to more sophisticated agent architectures

This notebook uses implementations from [agents.py](https://github.com/aimacode/aima-python/blob/master/agents.py) module. Let's start by importing everything from agents module.

In [ ]:
from agents import *
from notebook import psource

## CONTENTS

* Overview
* Agent
* Environment
* Simple Agent and Environment
* Agents in a 2-D Environment
* Wumpus Environment

## OVERVIEW

### What is an Intelligent Agent?

An **agent**, as defined in Section 2.1 of AIMA, is anything that can:
1. **Perceive** its environment through **sensors**
2. **Act** upon that environment through **actuators** 
3. Make decisions based on its **agent program**

**Mathematical Definition**: An agent can be described as a function that maps percept sequences to actions:
```
f: P* → A
```
Where P* is the set of all possible percept sequences, and A is the set of possible actions.

### Real-World Examples
- **Humans**: Eyes/ears (sensors) → Brain (program) → Hands/voice (actuators)
- **Robots**: Cameras/LIDAR (sensors) → CPU (program) → Motors/servos (actuators)  
- **Software agents**: Network data (sensors) → Algorithm (program) → API calls (actuators)

### The Agent-Environment Loop
```
Environment → Percepts → Agent → Actions → Environment
```

This notebook will demonstrate how to implement this fundamental AI concept step by step, starting with simple reactive agents and progressing to more complex behaviors.

## AGENT

Let us now see how we define an agent. Run the next cell to see how `Agent` is defined in agents module.

In [ ]:
psource(Agent)

### Understanding the Agent Class Structure

The `Agent` class provides the foundation for all intelligent agents. Let's examine its key components:

#### Constructor: `__init__(self, program=None)`
The constructor initializes the agent's **internal state** with these crucial attributes:

* **`alive`**: Tracks agent vitality (important for survival-based environments)
* **`bump`**: Collision detection (helps agent understand environment boundaries)  
* **`holding`**: Memory of carried objects (enables goal-directed behavior)
* **`performance`**: Quantitative evaluation metric (measures agent effectiveness)
* **`program`**: The "brain" of the agent - maps percepts to actions

> **Key Insight**: The `program` parameter is the most critical - it defines the agent's **behavior policy**. If no program is provided, the agent will ask for human input (manual control mode).

#### Method: `can_grab(self, thing)`
This method defines the agent's **physical capabilities** - what objects it can interact with in the environment. By default, agents cannot carry anything (like our "blind dog" example).

### Why These Attributes Matter
These attributes represent the **PEAS framework** components:
- **P**erformance: `performance` attribute
- **E**nvironment: Handled by separate Environment class  
- **A**ctuators: Methods like movement and actions
- **S**ensors: `percept()` method input

## ENVIRONMENT
Now, let us see how environments are defined. Running the next cell will display an implementation of the abstract `Environment` class.

In [ ]:
psource(Environment)

### Understanding the Environment Class

The `Environment` class models the world in which our agents operate. It has many methods, but let's focus on the essential ones for understanding agent-environment interaction:

#### Core Environment Methods:

* **`thing_classes(self)`**: Defines what types of objects can exist in this environment (environment constraints)

* **`add_thing(self, thing, location=None)`**: Places objects in the environment at specific locations

* **`run(self, steps)`**: **The main simulation loop** - executes the agent-environment interaction for a specified number of time steps

* **`is_done(self)`**: Termination condition - determines when the simulation should end

#### Critical Abstract Methods (Must Be Implemented):
Every Environment subclass **must** implement these two methods to enable the agent-environment loop:

* **`percept(self, agent)`**: **Sensor function** - returns what the agent can "see" at its current location
  - Input: The agent requesting percepts
  - Output: List of perceivable objects/stimuli
  - This is the agent's "sensory input"

* **`execute_action(self, agent, action)`**: **Actuator function** - processes agent actions and updates world state  
  - Input: The agent and its chosen action
  - Output: Changes to environment state
  - This is how agent actions "affect the world"

### The Agent-Environment Interaction Cycle:
```
1. Environment calls agent.program(percepts)
2. Agent.program returns an action  
3. Environment.execute_action(agent, action) 
4. Environment state changes
5. Repeat until is_done() returns True
```

This cycle implements the fundamental **perception-action loop** that drives all intelligent behavior.

## SIMPLE AGENT AND ENVIRONMENT

Let's implement our first intelligent agent using the **Simple Reflex Agent** architecture from AIMA Chapter 2. 

### Why a "Blind Dog"?
We're creating a "blind dog" agent to illustrate key AI concepts:
- **Limited perception**: Can only sense objects at its exact location (like touch)
- **Simple behavior**: Basic stimulus-response reactions  
- **Goal-directed**: Seeks food and water (survival needs)

This demonstrates a **reactive agent** - one that chooses actions based only on current percepts, without memory of past states.

In [ ]:
class BlindDog(Agent):
    def eat(self, thing):
        # Action: Consume food when available
        print("Dog: Ate food at {}.".format(self.location))
            
    def drink(self, thing):
        # Action: Consume water when available  
        print("Dog: Drank water at {}.".format( self.location))

# Create our first agent instance
dog = BlindDog()

### Agent State Inspection

What we have just created is a dog agent who can only perceive objects at its exact location (since it's "blind" to distant objects), and can perform eating or drinking actions. 

Let's check if our agent is alive and ready for action...

In [ ]:
# Check the agent's initial state
print(dog.alive)

![Cool dog](https://gifgun.files.wordpress.com/2015/07/wpid-wp-1435860392895.gif)
This is our dog. How cool is he? Well, he's hungry and needs to go search for food. For him to do this, we need to give him a program. But before that, let's create a park for our dog to play in.

### ENVIRONMENT - Park

Now we need to create an **environment** for our agent to inhabit. A park serves as an excellent example because:

1. **Observable**: Our dog can sense objects at its location
2. **Deterministic**: Actions have predictable results  
3. **Sequential**: Actions affect future percepts
4. **Dynamic**: Objects can be consumed (food/water disappears)

Since `Environment` is an abstract class, we must create our own concrete subclass that implements the required methods.

In [ ]:
# Define the objects that can exist in our environment
class Food(Thing):
    pass

class Water(Thing):
    pass

class Park(Environment):
    def percept(self, agent):
        '''SENSOR FUNCTION: Return list of objects at agent's current location
        This implements the agent's sensory capabilities - what it can "feel" or detect'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''ACTUATOR FUNCTION: Process agent's action and update environment state
        This implements how agent actions change the world'''
        if action == "move down":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.movedown()
        elif action == "eat":
            # Find food at current location
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): # Agent attempts to eat first food item
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) # Remove consumed food from environment
        elif action == "drink":
            # Find water at current location
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): # Agent attempts to drink first water item
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) # Remove consumed water from environment

    def is_done(self):
        '''TERMINATION CONDITION: Simulation ends when no food/water remains or agent dies
        This prevents our cute dog from starving by ending simulation before resources are exhausted'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles

### PROGRAM - BlindDog Enhanced

Now that we have a `Park` environment, we need to enhance our `BlindDog` with:
1. **Mobility**: Ability to move through the environment
2. **Decision-making**: Logic to determine when eating/drinking is successful
3. **Spatial awareness**: Track location in the 1D park space

This represents an upgrade from a purely reactive agent to one with basic **state-based behavior**.

In [ ]:
class BlindDog(Agent):
    location = 1  # Agent's position in 1D space (starting location)
    
    def movedown(self):
        # ACTUATOR: Move to next location in linear space
        self.location += 1
        
    def eat(self, thing):
        '''EATING BEHAVIOR: Attempt to consume food
        Returns True if successful, False otherwise'''
        if isinstance(thing, Food):
            return True  # Successfully ate food
        return False     # Cannot eat non-food items
    
    def drink(self, thing):
        '''DRINKING BEHAVIOR: Attempt to consume water
        Returns True if successful, False otherwise'''
        if isinstance(thing, Water):
            return True  # Successfully drank water
        return False     # Cannot drink non-water items

### Implementing the Agent Program - The "Brain" of Our Agent

Now we implement the **agent program** - the core decision-making function that maps percepts to actions. This represents the agent's **policy** or **behavior strategy**.

Our program implements a **Simple Reflex Agent** with the following condition-action rules:

<table>
    <tr>
        <td><b>Percept:</b> </td>
        <td>Feel Food </td>
        <td>Feel Water</td>
        <td>Feel Nothing</td>
   </tr>
   <tr>
       <td><b>Action:</b> </td>
       <td>eat</td>
       <td>drink</td>
       <td>move down</td>
   </tr>
</table>

**Key Insight**: This is a **greedy** policy - the agent always chooses immediate consumption over exploration. This works well when resources are guaranteed to be found by linear search.

**Algorithm Type**: This implements a simple **lookup table** or **condition-action rules** - the most basic form of AI decision making.

In [ ]:
def program(percepts):
    '''AGENT PROGRAM: The decision-making function that maps percepts to actions
    This is the "brain" of our agent - implements Simple Reflex Agent behavior
    
    Args:
        percepts: List of objects/stimuli the agent can sense at current location
    Returns:
        action: String representing the chosen action
    '''
    # Priority 1: Consume resources if available (survival behavior)
    for p in percepts:
        if isinstance(p, Food):
            return 'eat'      # Immediate consumption of food
        elif isinstance(p, Water):
            return 'drink'    # Immediate consumption of water
    
    # Priority 2: Explore environment if no resources detected  
    return 'move down'        # Continue searching

### Running Our First Agent Simulation

Now let's create a complete agent-environment system and observe the **perception-action loop** in action. We'll set up a scenario with strategically placed resources to test our agent's behavior.

In [ ]:
# SIMULATION SETUP: Create environment and populate with agent and resources
park = Park()
dog = BlindDog(program)   # Attach our decision-making program to the agent
dogfood = Food()
water = Water()

# ENVIRONMENT INITIALIZATION: Place objects at specific locations
park.add_thing(dog, 1)      # Dog starts at location 1
park.add_thing(dogfood, 5)  # Food at location 5  
park.add_thing(water, 7)    # Water at location 7

# RUN SIMULATION: Execute 5 time steps of agent-environment interaction
park.run(5)

### Analyzing Agent Behavior

**Observation**: The dog moved from location 1 → 4 over 4 steps, then ate food at location 5 in the 5th step.

**Analysis**: This demonstrates the **exploration-exploitation trade-off**:
- **Exploration phase**: Steps 1-4, agent searches environment  
- **Exploitation phase**: Step 5, agent consumes discovered resource

Let's continue the simulation to see what happens next...

In [ ]:
# CONTINUE SIMULATION: Run 5 more steps to observe complete behavior
park.run(5)

### Understanding Simulation Termination

**Perfect!** Notice how the simulation **automatically stopped** after the dog consumed the water. This demonstrates an important AI concept: **goal achievement** and **termination conditions**.

**Why did it stop?** Our `is_done()` method detected that all consumable resources were exhausted, meeting our defined success criteria.

Let's test the agent's **persistence** by adding more resources and observing continued exploration...

In [ ]:
# DYNAMIC ENVIRONMENT: Add resources during simulation to test adaptability
park.add_thing(water, 15)  # Place water at distant location 15
park.run(10)               # Run longer simulation to test persistence

### Key Learnings from 1D Agent Simulation

We've successfully implemented and tested a **Simple Reflex Agent** that demonstrates:
1. **Perception-Action Loop**: Continuous sensing and responding
2. **Goal-Directed Behavior**: Seeking survival resources  
3. **Environment Interaction**: Modifying world state through actions
4. **Termination Conditions**: Recognizing task completion

However, this was a simplified case with **limited complexity**. Real-world AI agents face more challenging environments!

## AGENTS IN A 2D ENVIRONMENT

### Why Upgrade to 2D?

Moving from 1D to 2D space introduces several important AI challenges:

1. **Increased State Space**: Exponentially more possible locations
2. **Navigation Complexity**: Need for spatial reasoning and pathfinding  
3. **Directional Awareness**: Agents must track orientation (North, South, East, West)
4. **Boundary Handling**: Collision detection and environment limits
5. **Visual Representation**: Graphics help us understand agent behavior

### Introducing GraphicEnvironment

To visualize our 2D world, we'll use `GraphicEnvironment` which adds:

- **Coordinate System**: Standard X-Y plane positioning (4th quadrant indexing)
- **Color Coding**: RGB visual representation of objects (Red=Agent, Blue=Water, Orange=Food)  
- **Boundary Fencing**: Automatic collision detection with environment edges
- **Real-time Visualization**: Watch agents move and interact graphically

**Safety Feature**: The `is_inbounds()` function prevents our blind dog from wandering outside the park - it's dangerous out there!

Let's upgrade our simple Park environment to support 2D navigation...

In [ ]:
class Park2D(GraphicEnvironment):
    def percept(self, agent):
        '''SENSOR FUNCTION: Return objects at agent's current [x,y] location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''ACTUATOR FUNCTION: Process agent actions in 2D space'''
        if action == "move down":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.movedown()  # Move in Y direction
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]):
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])  # Remove consumed food
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]):
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])  # Remove consumed water
                    
    def is_done(self):
        '''TERMINATION: End when no resources remain or agent dies'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles

class BlindDog(Agent):
    location = [0,1]              # 2D coordinate [x, y] position
    direction = Direction("down") # Track which way agent is facing
    
    def movedown(self):
        # Move in positive Y direction (down in 4th quadrant)
        self.location[1] += 1
        
    def eat(self, thing):
        '''Consume food if present at current location'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        '''Consume water if present at current location'''
        if isinstance(thing, Water):
            return True
        return False

### Testing 2D Environment with Visual Feedback

Now let's test our upgraded 2D park with the same dog behavior, but enhanced with **color-coded visualization**:
- **Red (200,0,0)**: Our blind dog agent
- **Blue (0,200,200)**: Water resources  
- **Orange (230,115,40)**: Food resources

This visual feedback helps us understand agent behavior patterns and environment dynamics.

In [ ]:
# CREATE 2D ENVIRONMENT: 5x20 grid with color-coded visualization
park = Park2D(5,20, color={'BlindDog': (200,0,0), 'Water': (0, 200, 200), 'Food': (230, 115, 40)}) 
dog = BlindDog(program)       # Use same decision-making program
dogfood = Food()
water = Water()

# STRATEGIC PLACEMENT: Resources along the dog's linear path
park.add_thing(dog, [0,1])       # Start at [0,1] 
park.add_thing(dogfood, [0,5])   # Food directly ahead at [0,5]
park.add_thing(water, [0,7])     # Water further along at [0,7] 
morewater = Water()
park.add_thing(morewater, [0,15]) # Additional water at [0,15]

print("BlindDog starts at (0,1) facing downwards, let's see if he can find any food!")
park.run(20)  # Longer simulation for 2D environment

### Limitation Analysis: Why Our Dog Needs Better Navigation

**Observation**: The graphics clearly show our blind dog **isn't utilizing the 2D space effectively** - it only moves in straight lines!

**Problem Identified**: Our current agent has a **limited action repertoire** - it can only move "down" but cannot:
- Turn left or right
- Move in different directions  
- Explore the full 2D environment
- Handle obstacles or boundaries

### Solution: Upgrade to EnergeticBlindDog

Let's enhance our agent with **multi-directional navigation capabilities**:

1. **Rotation Actions**: Turn left/right to change facing direction
2. **Forward Movement**: Move in the currently facing direction
3. **Collision Detection**: Respond to boundary encounters with direction changes
4. **Stochastic Behavior**: Random exploration when no resources are detected

This represents an evolution from **Simple Reflex Agent** to **Randomized Agent** with enhanced mobility.

### New Behavioral Strategy

<table>
    <tr>
        <td><b>Percept:</b> </td>
        <td>Feel Food </td>
        <td>Feel Water</td>
        <td>Feel Nothing</td>
   </tr>
   <tr>
       <td><b>Action:</b> </td>
       <td>eat</td>
       <td>drink</td>
       <td>
       <table>
           <tr>
               <td><b>Situation:</b></td>
               <td>At Boundary</td>
               <td>Free Space</td>
           </tr>
           <tr>
               <td><b>Action Distribution:</b></td>
               <td>Turn Left (50%) / Turn Right (50%)</td>
               <td>Turn Left (25%) / Turn Right (25%) / Move Forward (50%)</td>
           </tr>
       </table>
       </td>
   </tr>
</table>

**Key Innovation**: **Probabilistic action selection** introduces **exploration behavior** while **collision avoidance** prevents the agent from getting stuck at boundaries.

In [ ]:
from random import choice

class EnergeticBlindDog(Agent):
    location = [0,1]              # Starting position in 2D space
    direction = Direction("down") # Current facing direction
    
    def moveforward(self, success=True):
        '''ENHANCED MOBILITY: Move forward in currently facing direction
        Args:
            success: Whether movement is allowed (used for collision prevention)
        '''
        if not success:
            return  # Collision prevention - don't move if blocked
            
        # DIRECTIONAL MOVEMENT: Update position based on current orientation
        if self.direction.direction == Direction.R:      # Moving Right
            self.location[0] += 1
        elif self.direction.direction == Direction.L:    # Moving Left  
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:    # Moving Down
            self.location[1] += 1
        elif self.direction.direction == Direction.U:    # Moving Up
            self.location[1] -= 1
    
    def turn(self, d):
        '''ROTATION CAPABILITY: Change facing direction'''
        self.direction = self.direction + d
        
    def eat(self, thing):
        '''Resource consumption behavior - unchanged from previous version'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        '''Resource consumption behavior - unchanged from previous version'''
        if isinstance(thing, Water):
            return True
        return False
        
def program(percepts):
    '''ENHANCED AGENT PROGRAM: Implements probabilistic exploration with collision avoidance
    
    Decision Hierarchy:
    1. SURVIVAL: Consume resources if available (highest priority)
    2. AVOIDANCE: Turn when collision detected (safety)  
    3. EXPLORATION: Random movement in free space (discovery)
    '''
    
    # PRIORITY 1: Resource consumption (deterministic behavior)
    for p in percepts:
        if isinstance(p, Food):
            return 'eat'    # Immediate food consumption
        elif isinstance(p, Water):
            return 'drink'  # Immediate water consumption
        
        # PRIORITY 2: Collision avoidance (safety behavior)
        if isinstance(p,Bump):  # Boundary detected ahead
            turn = False
            choice = random.choice((1,2))  # 50-50 turn decision
        else:
            # PRIORITY 3: Exploration (stochastic behavior)
            choice = random.choice((1,2,3,4))  # 25% left, 25% right, 50% forward
            
    # ACTION SELECTION: Convert choice to actual command
    if choice == 1:
        return 'turnright'   # Rotate clockwise
    elif choice == 2:
        return 'turnleft'    # Rotate counter-clockwise  
    else:
        return 'moveforward' # Continue in current direction

### ENVIRONMENT - Enhanced Park2D

Our environment must now support the agent's expanded action repertoire. Key enhancements include:

1. **Predictive Collision Detection**: Check if next move would hit boundary
2. **Multi-Action Processing**: Handle turn and movement commands
3. **Boundary Feedback**: Provide "Bump" percepts when collision is imminent

**Safety Design**: We prevent boundary violations rather than allowing them and recovering - this is more realistic for physical agents.

In [ ]:
class Park2D(GraphicEnvironment):
    def percept(self, agent):
        '''ENHANCED SENSOR FUNCTION: Provide location contents + collision warnings'''
        things = self.list_things_at(agent.location)
        
        # PREDICTIVE COLLISION DETECTION: Check if forward movement would hit boundary
        loc = copy.deepcopy(agent.location)  # Calculate target location
        
        # Determine next position based on current facing direction
        if agent.direction.direction == Direction.R:
            loc[0] += 1  # Moving right increases X
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1  # Moving left decreases X
        elif agent.direction.direction == Direction.D:
            loc[1] += 1  # Moving down increases Y  
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1  # Moving up decreases Y
            
        # BOUNDARY CHECK: Add Bump percept if next move would be out of bounds
        if not self.is_inbounds(loc):
            things.append(Bump())  # Warning: collision ahead!
            
        return things
    
    def execute_action(self, agent, action):
        '''ENHANCED ACTUATOR FUNCTION: Process rotational and translational actions'''
        if action == 'turnright':
            # ROTATION ACTION: Change direction clockwise
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
        elif action == 'turnleft':
            # ROTATION ACTION: Change direction counter-clockwise  
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        elif action == 'moveforward':
            # TRANSLATION ACTION: Move in facing direction
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.moveforward()
        elif action == "eat":
            # CONSUMPTION ACTION: Eat food at current location
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]):
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])  # Remove consumed food
        elif action == "drink":
            # CONSUMPTION ACTION: Drink water at current location
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]):
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])  # Remove consumed water
                    
    def is_done(self):
        '''TERMINATION CONDITION: End simulation when resources exhausted or agent dies'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles

### Testing Enhanced 2D Navigation

Now let's observe our **EnergeticBlindDog** with full 2D mobility! This simulation will demonstrate:

1. **Stochastic Exploration**: Random walk behavior in open areas
2. **Collision Avoidance**: Boundary detection and evasion  
3. **Resource Discovery**: Finding scattered food and water
4. **Emergent Behavior**: Complex patterns from simple rules

Watch how **random exploration** can effectively cover the 2D space, even without sophisticated pathfinding algorithms.

In [ ]:
# ENHANCED 2D SIMULATION: Smaller space with scattered resources
park = Park2D(5,5, color={'EnergeticBlindDog': (200,0,0), 'Water': (0, 200, 200), 'Food': (230, 115, 40)})
dog = EnergeticBlindDog(program)  # Use enhanced agent with rotational capabilities

# STRATEGIC RESOURCE DISTRIBUTION: Test 2D exploration ability
dogfood = Food()
water = Water()
park.add_thing(dog, [0,0])        # Agent starts at corner [0,0]
park.add_thing(dogfood, [1,2])    # Food requires 2D navigation to reach
park.add_thing(water, [0,1])      # Water nearby for quick discovery
morewater = Water()
morefood = Food()
park.add_thing(morewater, [2,4])  # Distant water tests persistence  
park.add_thing(morefood, [4,3])   # Far corner food tests exploration

print("Enhanced dog started at [0,0], facing down. Let's see if 2D exploration finds the scattered resources!")
park.run(20)  # Extended simulation to observe exploration patterns

### Reflection Questions

**Before proceeding to the Wumpus World, consider these questions:**

1. **Efficiency vs. Exploration**: How does random exploration compare to systematic search? What are the trade-offs?

2. **Scalability**: How would our agent perform in larger environments (100x100 grid)? 

3. **Learning**: Could our agent improve its performance by remembering visited locations?

4. **Cooperation**: How might multiple agents coordinate in the same environment?

5. **Real-world Applications**: Where do you see similar random exploration strategies used in robotics or AI systems?

---

## Wumpus Environment

### Introduction to a Classic AI Challenge

The **Wumpus World** is one of the most famous environments in AI literature, introduced in Russell & Norvig's textbook. It represents a significant step up in complexity from our simple park environment:

**New Challenges:**
- **Partial Observability**: Agent cannot see the entire environment
- **Danger**: Deadly pits and the Wumpus creature can kill the agent
- **Uncertainty**: Indirect evidence (stench, breeze) must be reasoned about
- **Risk Assessment**: Agent must balance exploration vs. safety
- **Goal-Oriented**: Specific objective (find gold and return safely)

**Why Important**: This environment requires **logical reasoning**, **knowledge representation**, and **planning under uncertainty** - core AI capabilities beyond simple reactive behavior.

In [ ]:
from ipythonblocks import BlockGrid
from agents import *

# WUMPUS WORLD COLOR SCHEME: Visual representation of environment elements  
color = {"Breeze": (225, 225, 225),    # Light gray - indicates nearby pit
        "Pit": (0,0,0),                # Black - deadly obstacle
        "Gold": (253, 208, 23),         # Bright yellow - goal object
        "Glitter": (253, 208, 23),      # Yellow - indicates gold nearby
        "Wumpus": (43, 27, 23),         # Dark brown - dangerous creature
        "Stench": (128, 128, 128),      # Gray - indicates Wumpus nearby  
        "Explorer": (0, 0, 255),        # Blue - our brave agent
        "Wall": (44, 53, 57)            # Dark gray - environment boundary
        }

def program(percepts):
    '''INTERACTIVE AGENT PROGRAM: Human-controlled decision making
    This allows you to manually control the agent and experience the challenges
    of reasoning under uncertainty in the Wumpus World'''
    print(percepts)  # Display current sensory information
    return input()   # Wait for human input (action command)

# CREATE WUMPUS ENVIRONMENT: 7x7 grid world with hidden dangers
w = WumpusEnvironment(program, 7, 7)         
grid = BlockGrid(w.width, w.height, fill=(123, 234, 123))  # Green background

def draw_grid(world):
    '''VISUALIZATION FUNCTION: Render current world state as colored grid'''
    global grid
    grid[:] = (123, 234, 123)  # Reset to background color
    
    # Color each cell based on its contents
    for x in range(0, len(world)):
        for y in range(0, len(world[x])):
            if len(world[x][y]):
                # Use color of topmost object in each cell
                grid[y, x] = color[world[x][y][-1].__class__.__name__]

def step():
    '''SIMULATION STEP: Update display and advance one time step'''
    global grid, w
    draw_grid(w.get_world())  # Render current state
    grid.show()               # Display visual grid
    w.step()                  # Process one agent action

In [ ]:
# START WUMPUS WORLD SIMULATION
# Execute this cell to begin your adventure!
# Available actions: 'Forward', 'TurnLeft', 'TurnRight', 'Shoot', 'Grab', 'Climb'
step()

### Wumpus World Challenge Instructions

**Your Mission**: Navigate the dangerous cave system, find the gold, and return safely to the entrance.

**Available Actions:**
- `Forward` - Move one square in the facing direction
- `TurnLeft` - Rotate 90° counter-clockwise  
- `TurnRight` - Rotate 90° clockwise
- `Shoot` - Fire arrow in facing direction (kills Wumpus, only one arrow available)
- `Grab` - Pick up gold if present in current square
- `Climb` - Exit cave (only works at entrance square [1,1])

**Percepts and Their Meanings:**
- **Stench**: Wumpus is in an adjacent square (dangerous!)
- **Breeze**: Pit is in an adjacent square (deadly!)  
- **Glitter**: Gold is in current square (goal!)
- **Bump**: Tried to move into wall (blocked)
- **Scream**: Your arrow killed the Wumpus (success!)

**Strategic Thinking**: Use logical reasoning to deduce safe squares. For example:
- If you sense a breeze, adjacent squares may contain pits
- If no breeze, adjacent squares are guaranteed safe from pits
- The Wumpus location can be triangulated using stench percepts from multiple positions

**Challenge**: Try to complete the mission with minimal exploration - can you find the gold without visiting every square?

## Summary and Next Steps

### What We've Learned

Through this notebook, we've explored the fundamental concepts of intelligent agents:

1. **Agent Architecture**: Understanding the sensor-actuator-program structure
2. **Environment Design**: Creating worlds for agents to inhabit and interact with
3. **Behavioral Evolution**: Progressing from simple reflex to probabilistic exploration
4. **Dimensional Complexity**: Scaling from 1D to 2D navigation challenges  
5. **Visual Feedback**: Using graphics to understand and debug agent behavior
6. **Classic Challenges**: Experiencing the Wumpus World's reasoning requirements

### Key AI Concepts Demonstrated

- **Perception-Action Loop**: The fundamental cycle of intelligent behavior
- **State Representation**: How agents track their position and orientation  
- **Decision Making**: From deterministic rules to stochastic policies
- **Collision Avoidance**: Safety mechanisms for physical agents
- **Exploration vs. Exploitation**: Balancing discovery with resource utilization
- **Reasoning Under Uncertainty**: Logic and inference in partially observable environments

**Congratulations!** You now understand the core principles of intelligent agents and are ready to explore more advanced AI architectures and algorithms.